In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Standard_Scaler_result.csv',encoding='cp949')

In [4]:

from sklearn.model_selection import train_test_split

In [9]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','KIND_JOB_A', 'KIND_JOB_B', 'KIND_JOB_C', 'KIND_JOB_D',
       'KIND_JOB_E', 'KIND_JOB_F', 'KIND_JOB_G', 'KIND_JOB_H', 'KIND_JOB_I',
       'KIND_JOB_J', 'KIND_JOB_K', 'KIND_JOB_L', 'KIND_JOB_M', 'KIND_JOB_N',
       'KIND_JOB_O', 'KIND_JOB_P', 'KIND_JOB_Q', 'KIND_JOB_R', 'KIND_JOB_S',
       'KIND_JOB_T']]
act = data['ACTCD']

In [10]:

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 41) (11736, 41) (27381,) (11736,)


In [11]:
'''
1. ENN
2. ADASYN
3. BORDERLINE SMOTE
4. Tomeklinks
5 SOTEENN
6.SMOTETOMEK
'''
from imblearn.over_sampling import BorderlineSMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
data_sets={}
enn = EditedNearestNeighbours()
ada = ADASYN(random_state=42)
smte = SMOTEENN(random_state=42)
smtt = SMOTETomek(random_state=42)
tk = TomekLinks()
sm = BorderlineSMOTE(random_state=42)
enn_under_train_x,enn_under_train_y = enn.fit_resample(train_x,train_y)
ada_over_train_x, ada_over_train_y = ada.fit_resample(train_x,train_y)
smte_combine_train_x,smte_combine_train_y = smte.fit_resample(train_x,train_y)
smtt_combine_train_x,smtt_combine_train_y = smtt.fit_resample(train_x,train_y)
tk_under_train_x,tk_under_train_y = tk.fit_resample(train_x,train_y)
sm_over_train_x, sm_over_train_y = sm.fit_resample(train_x,train_y)
data_sets['enn']=[enn_under_train_x,enn_under_train_y]
data_sets['ada']=[ada_over_train_x,ada_over_train_y]
data_sets['smte']=[smte_combine_train_x,smte_combine_train_y]
data_sets['smtt']=[smtt_combine_train_x,smtt_combine_train_y]
data_sets['tk']=[tk_under_train_x,tk_under_train_y]
data_sets['sm']=[sm_over_train_x,sm_over_train_y]
'''
over_train_x : 학습용 독립변수
over_train_y : 학습용 종속변수
'''
keys_list= data_sets.keys()
accuracies = []
matrix_kd_tree_uniform_BOST = []

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix, classification_report


In [13]:
for i in keys_list:
    model = RandomForestClassifier(n_estimators=200,max_depth=50,max_features=1)
    model.fit(data_sets[i][0], data_sets[i][1])
                            
                    # 결정계수
    relation_square = model.score(data_sets[i][0], data_sets[i][1])
    print('결정계수 : ', relation_square)
                    
                    # 예측
    pred_y = model.predict(test_x)
    pr = precision_score(y_true=test_y,y_pred=pred_y)
    r = recall_score(y_true=test_y,y_pred=pred_y)
    f1 = f1_score(y_true=test_y,y_pred=pred_y)
    rca = roc_auc_score(y_true=test_y,y_score=pred_y)
    cfm = confusion_matrix(y_true=test_y,y_pred=pred_y)
    matrix_kd_tree_uniform_BOST.append((i,pr,r,f1,rca,model.score(test_x, test_y),cfm))
                # 정확도 확인
    print(i)
    print('정확도 : ', metrics.accuracy_score(test_y, pred_y))
    print(classification_report(test_y, pred_y, target_names=['사고보증', '정상보증']))
    print('\n')

결정계수 :  0.9995086653564409
enn
정확도 :  0.9238241308793456
              precision    recall  f1-score   support

        사고보증       0.93      0.99      0.96     10666
        정상보증       0.76      0.24      0.37      1070

    accuracy                           0.92     11736
   macro avg       0.84      0.62      0.66     11736
weighted avg       0.91      0.92      0.91     11736



결정계수 :  0.9989307186377759
ada
정확도 :  0.9104464894342195
              precision    recall  f1-score   support

        사고보증       0.93      0.97      0.95     10666
        정상보증       0.52      0.28      0.36      1070

    accuracy                           0.91     11736
   macro avg       0.72      0.63      0.66     11736
weighted avg       0.89      0.91      0.90     11736



결정계수 :  1.0
smte
정확도 :  0.84986366734833
              precision    recall  f1-score   support

        사고보증       0.94      0.89      0.92     10666
        정상보증       0.29      0.46      0.36      1070

    accuracy           

In [16]:
matrix_kd_tree_uniform_BOST

[('enn',
  0.7463556851311953,
  0.23925233644859814,
  0.36234961075725414,
  0.6155477883255553,
  0.923227675528289,
  array([[10579,    87],
         [  814,   256]], dtype=int64)),
 ('ada',
  0.48406676783004554,
  0.29813084112149535,
  0.3689994216310006,
  0.633126924404738,
  0.9070381731424676,
  array([[10326,   340],
         [  751,   319]], dtype=int64)),
 ('smte',
  0.28896672504378285,
  0.46261682242990654,
  0.3557312252964427,
  0.6742110926325419,
  0.8472222222222222,
  array([[9448, 1218],
         [ 575,  495]], dtype=int64)),
 ('smtt',
  0.48829953198127923,
  0.2925233644859813,
  0.36586791350087666,
  0.6308857212454283,
  0.9075494205862304,
  array([[10338,   328],
         [  757,   313]], dtype=int64)),
 ('tk',
  0.9015748031496063,
  0.21401869158878506,
  0.34592145015105746,
  0.6058373975476271,
  0.926209952283572,
  array([[10641,    25],
         [  841,   229]], dtype=int64)),
 ('sm',
  0.5269360269360269,
  0.2925233644859813,
  0.376201923076923